In [7]:
from sys import path


path.append("/opt/configs/ramjet")

from prd import *

In [72]:
s = requests.Session()
s.headers

{'User-Agent': 'python-requests/2.25.1', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive'}

In [8]:
# -*- coding=utf-8
# appid 已在配置中移除,请在参数 Bucket 中带上 appid。Bucket 由 BucketName-APPID 组成
# 1. 设置用户配置, 包括 secretId，secretKey 以及 Region
from qcloud_cos import CosConfig
from qcloud_cos import CosS3Client
import sys
import logging

logging.basicConfig(level=logging.INFO, stream=sys.stdout)

secret_id = TENCENT_CLOUD_SECRET_ID      # 替换为用户的 secretId
secret_key = TENCENT_CLOUD_SECRET_KEY      # 替换为用户的 secretKey
region = TENCENT_CLOUD_REGION     # 替换为用户的 Region
token = None                # 使用临时密钥需要传入 Token，默认为空，可不填
scheme = 'https'            # 指定使用 http/https 协议来访问 COS，默认为 https，可不填

config = CosConfig(Region=region, SecretId=secret_id, SecretKey=secret_key, Token=token, Scheme=scheme)
# 2. 获取客户端对象
client = CosS3Client(config)
# 参照下文的描述。或者参照 Demo 程序，详见 https://github.com/tencentyun/cos-python-sdk-v5/blob/master/qcloud_cos/demo.py

In [9]:
client.list_buckets()

{'Owner': {'ID': 'qcs::cam::uin/270620769:uin/270620769',
  'DisplayName': '270620769'},
 'Buckets': {'Bucket': [{'Name': 'watermark-input-1251663498',
    'Location': 'ap-shanghai',
    'CreationDate': '2021-03-10T06:08:08Z'}]}}

In [76]:
import tempfile

fp = tempfile.TemporaryFile()
fp.

57

In [18]:
# 生成盲水印

import json
from os import path


def upload_image(fpath: str):
    _,fname = path.split(fpath)
    with open(fpath, 'rb') as fp:
        resp = client.put_object(
            Bucket='watermark-input-1251663498',
            Body=fp,
            Key=fname,
            StorageClass='STANDARD',
            EnableMD5=False,
            **{
                "PicOperations": json.dumps({
                    "is_pic_info": 0,
                    "rules": [{
                        "fileid": f"/processed/{fname}",
                        "rule": "watermark/3/type/3/image/xxx/text/laisky/level/1"
                    }]
                })
            }
        )
        
    return resp
        
upload_image('/root/manmanmai.jpg')

INFO:qcloud_cos.cos_client:put object, url=:https://watermark-input-1251663498.cos.ap-shanghai.myqcloud.com/manmanmai.jpg ,headers=:{'x-cos-storage-class': 'STANDARD', 'Pic-Operations': '{"is_pic_info": 1, "rules": [{"fileid": "/processed/manmanmai.jpg", "rule": "watermark/3/type/3/image/xxx/text/laisky/level/1"}]}'}


{'Content-Type': 'application/xml',
 'Content-Length': '822',
 'Connection': 'keep-alive',
 'Date': 'Thu, 11 Mar 2021 02:38:08 GMT',
 'ETag': '"d5c405564a99eee6840495cc4374a841d3db01be"',
 'Server': 'tencent-cos',
 'x-cos-hash-crc64ecma': '15559843189058836064',
 'x-cos-request-id': 'NjA0OTgyOTBfMmViYzAzMDlfZjExOF9iYjAwZTg2'}

In [50]:
fname = "manmanmai.jpg"

json.dumps({
                    "is_pic_info": 0,
                    "rules": [{
                        "fileid": f"/processed/{fname}",
                        "rule": "watermark/3/type/3/text/dGVzdA=="
                    }]
                })

'{"is_pic_info": 0, "rules": [{"fileid": "/processed/manmanmai.jpg", "rule": "watermark/3/type/3/text/dGVzdA=="}]}'

In [41]:
# 下载时添加

import requests

resp = requests.get(
    url='http://watermark-input-1251663498.cos.ap-shanghai.myqcloud.com/manmanmai.jpg?watermark/3/type/3/text/dGVuY2VudCBjbG91ZA=='
)

with open('/root/watermark-demo.jpg', 'wb') as fp:
    fp.write(resp.content)
    
    

In [69]:
from base64 import b64encode, b64decode


print(b64decode('dGVzdA=='))
print(b64encode(("laisky270602769"*10).encode('utf8')))

b'test'
b'bGFpc2t5MjcwNjAyNzY5bGFpc2t5MjcwNjAyNzY5bGFpc2t5MjcwNjAyNzY5bGFpc2t5MjcwNjAyNzY5bGFpc2t5MjcwNjAyNzY5bGFpc2t5MjcwNjAyNzY5bGFpc2t5MjcwNjAyNzY5bGFpc2t5MjcwNjAyNzY5bGFpc2t5MjcwNjAyNzY5bGFpc2t5MjcwNjAyNzY5'


In [47]:
# 提取盲水印


def valid_image(fpath: str):
    _,fname = path.split(fpath)
    print(f"{fname=}")
    with open(fpath, 'rb') as fp:
        resp = client.put_object(
            Bucket='watermark-input-1251663498',
            Body=fp,
            Key=fname,
            StorageClass='STANDARD',
            EnableMD5=False,
            **{
                "PicOperations": json.dumps({
                    "is_pic_info": 1,
                    "rules": [{
                        "fileid": f"/extracted/{fname}",
                        "rule": "watermark/4/type/3/text/dGVuY2VudCBjbG91ZA=="
                    }]
                })
            }
        )
        
    return resp
        
valid_image('/root/watermark-demo.jpg')
# valid_image('/mnt/c/Users/ppcel/Downloads/manmanmai.jpg')

fname='watermark-demo.jpg'
INFO:qcloud_cos.cos_client:put object, url=:https://watermark-input-1251663498.cos.ap-shanghai.myqcloud.com/watermark-demo.jpg ,headers=:{'x-cos-storage-class': 'STANDARD', 'Pic-Operations': '{"is_pic_info": 1, "rules": [{"fileid": "/extracted/watermark-demo.jpg", "rule": "watermark/4/type/3/text/dGVuY2VudCBjbG91ZA=="}]}'}


{'Content-Type': 'application/xml',
 'Content-Length': '842',
 'Connection': 'keep-alive',
 'Date': 'Thu, 11 Mar 2021 04:16:09 GMT',
 'ETag': '"d81cd3103380c6bdbf277b00fd1ed0b9033d528e"',
 'Server': 'tencent-cos',
 'x-cos-hash-crc64ecma': '7555019929505311533',
 'x-cos-request-id': 'NjA0OTk5ODlfNzJiM2IxMDlfMzY3N2FfY2Q1NTFhMQ=='}